In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import spacy
import matplotlib.pyplot as plt
import time
import multiprocessing
import zipfile
from spacy.tokenizer import Tokenizer
from shutil import rmtree
import time


base_loc = "/home/alok/data/"
raw_zip_file = "s3_data/kaggle_coleridge/raw/coleridgeinitiative-show-us-the-data.zip"
unzip_loc = "s3_data/kaggle_coleridge/processed/unzipped/"

test_folder = "s3_data/kaggle_coleridge/processed/unzipped/test/"
train_folder = "s3_data/kaggle_coleridge/processed/unzipped/train/"

train_y_file = "train.csv"
submission_file = "sample_submission.csv"

output_folder = "s3_data/kaggle_coleridge/processed/processing_1/"

nlp = spacy.load("en_core_web_sm")

2021-07-23 23:04:10.454385: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-23 23:04:10.454408: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# if not os.path.exists(os.path.join(base_loc, unzip_loc)):
#     os.makedirs(os.path.join(base_loc, unzip_loc))
# else:
#     rmtree(os.path.join(base_loc, unzip_loc))
#     os.makedirs(os.path.join(base_loc, unzip_loc))
    

# with zipfile.ZipFile(os.path.join(base_loc, raw_zip_file), 'r') as zip_ref:
#     zip_ref.extractall(os.path.join(base_loc, unzip_loc))

In [3]:
# def jaccard(str1, str2): 
#     a = set(str1.lower().split()) 
#     b = set(str2.lower().split())
#     c = a.intersection(b)
#     return float(len(c)) / (len(a) + len(b) - len(c))


# def clean_text(txt):
#     return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [4]:
def find_label_in_text(sent_tokens, label_tokens):
    max_loc = len(sent_tokens) - len(label_tokens)
    for i in range(0, max_loc):
        temp_tokens = sent_tokens[i:i+len(label_tokens)]
        
        if temp_tokens == label_tokens:
            return [i,  i-1+len(label_tokens)]
    return []


def get_tokens_loc(sent, sent_tokens):
    tokens_loc_start, tokens_loc_end = [], []
    
    start_search = 0
    for token in sent_tokens:
        temp_loc = sent.find(token, start_search)
        
        assert temp_loc != -1, "token <%s> not found in sentence <%s>"%(token, sent)
        tokens_loc_start.append(temp_loc)
        tokens_loc_end.append(temp_loc + len(token))
        
        start_search += len(token)
        
    return tokens_loc_start, tokens_loc_end
    
    
def get_sentences(ind, base_loc, train_folder, file_id, pub_title,dataset_title, dataset_label):

    
    print("processing %s started"%ind)
    temp_file_loc = os.path.join(base_loc, train_folder, file_id+".json")
    json_data = json.load(open(temp_file_loc, "r"))
    sentences = []
    
    label_doc = nlp(dataset_label)
    label_tokens = [str(token) for token in label_doc]
    
    for elem in json_data:
        temp_text = elem["text"]
        temp_doc = nlp(temp_text)
        sents = list(temp_doc.sents)
        for s_ind, sent in enumerate(sents):
            sent = str(sent)
            
            sent_doc = nlp(sent)
            sent_tokens = [str(token) for token in sent_doc]
            sent_tokens_start, sent_tokens_end = get_tokens_loc(sent, sent_tokens)
            
            locations = find_label_in_text(sent_tokens, label_tokens)
            
            if locations:
                sentences.append({
                    "file": file_id+ ".json",
                    "publication": pub_title,
                    "section": elem["section_title"],
                    "sentence_id": s_ind,
                    "sentence": sent,
                    "tokens": sent_tokens,
                    "dataset_title": dataset_title,
                    "label": dataset_label,
                    "label_start_tokens": locations[0],
                    "bael_end_tokens": locations[1],
                    "label_start": sent_tokens_start[locations[0]],
                    "label_end": sent_tokens_end[locations[1]]
                    
                })
            else:
                sentences.append({
                    "file": file_id+ ".json",
                    "publication": pub_title,
                    "section": elem["section_title"],
                    "sentence_id": s_ind,
                    "sentence": sent,
                    "tokens": sent_tokens,
                })
    print("Processing %s finished"%ind)
    return sentences


In [5]:
train_df = pd.read_csv(os.path.join(base_loc, unzip_loc,  train_y_file))
inputs = []
count = 0
for i in range(train_df.shape[0]):
    inputs.append((
        count,
        base_loc,
        train_folder,
        train_df.loc[i, "Id"],
        train_df.loc[i, "pub_title"],
        train_df.loc[i, "dataset_title"],
        train_df.loc[i, "dataset_label"]
    ))
    count += 1

print(len(inputs))
print(inputs[0])

19661
(0, '/home/alok/data/', 's3_data/kaggle_coleridge/processed/unzipped/train/', 'd0fa7568-7d8e-4db9-870f-f9c6f668c17b', 'The Impact of Dual Enrollment on College Degree Attainment: Do Low-SES Students Benefit?', 'National Education Longitudinal Study', 'National Education Longitudinal Study')


In [ ]:
inputs= inputs[:5000]

print("processing started")
st = round(time.time(), 0)

# ORDINARY LOOP
# output = [get_sentences(*i) for i in inputs]


# POOL
with multiprocessing.Pool(3) as pool:
    # apply_async
    pooled_output = [ pool.apply_async(get_sentences, i) for i in inputs]
    output = [p.get() for p in pooled_output]
    
    # map
#     output = pool.starmap(get_sentences, inputs)

# # PROCESS
# pools = []
# q = multiprocessing.Queue()
# print(1)
# for inp in inputs:
#     temp_inp = (inp[0], inp[1], inp[2], inp[3], inp[4], inp[5], inp[6], q)
#     p = multiprocessing.Process(target=get_sentences_process, args=temp_inp)
    
#     p.start()
#     pools.append(p)

# output = []
# print(2, len(pools))
# for p_ind, p in enumerate(pools):
#     output.append(q.get())
#     print("closing", p_ind)
#     p.join()
#     print("closed", p_ind)

en = round(time.time(), 0)
print("program completed in %s seconds"%(en-st))

print(len(output))
print(output[0])

processing started
processing 0 started
processing 1 started
processing 2 started
Processing 1 finished
processing 3 started
Processing 0 finished
processing 4 started
Processing 3 finished
processing 5 started
Processing 5 finished
processing 6 started
Processing 2 finished
processing 7 started
Processing 4 finished
processing 8 started
Processing 7 finished
processing 9 started
Processing 9 finished
processing 10 started
Processing 6 finished
processing 11 started
Processing 8 finished
processing 12 started
Processing 10 finished
processing 13 started
Processing 11 finished
processing 14 started
Processing 12 finished
processing 15 started
Processing 15 finished
processing 16 started
Processing 13 finished
processing 17 started
Processing 14 finished
processing 18 started
Processing 17 finished
processing 19 started
Processing 18 finished
processing 20 started
Processing 16 finished
processing 21 started
Processing 19 finished
processing 22 started
Processing 20 finished
processing 2

Processing 177 finished
processing 180 started
Processing 178 finished
processing 181 started
Processing 179 finished
processing 182 started
Processing 180 finished
processing 183 started
Processing 181 finished
processing 184 started
Processing 182 finished
processing 185 started
Processing 183 finished
processing 186 started
Processing 184 finished
processing 187 started
Processing 186 finished
processing 188 started
Processing 185 finished
processing 189 started
Processing 187 finished
processing 190 started
Processing 188 finished
processing 191 started
Processing 190 finished
processing 192 started
Processing 189 finished
processing 193 started
Processing 191 finished
processing 194 started
Processing 192 finished
processing 195 started
Processing 193 finished
processing 196 started
Processing 194 finished
processing 197 started
Processing 195 finished
processing 198 started
Processing 196 finished
processing 199 started
Processing 197 finished
processing 200 started
Processing 19

Processing 347 finished
processing 354 started
Processing 351 finished
processing 355 started
Processing 355 finished
processing 356 started
Processing 356 finished
processing 357 started
Processing 353 finished
processing 358 started
Processing 354 finished
processing 359 started
Processing 357 finished
processing 360 started
Processing 358 finished
processing 361 started
Processing 360 finished
processing 362 started
Processing 361 finished
processing 363 started
Processing 362 finished
processing 364 started
Processing 363 finished
processing 365 started
Processing 364 finished
processing 366 started
Processing 365 finished
processing 367 started
Processing 359 finished
processing 368 started
Processing 366 finished
processing 369 started
Processing 368 finished
processing 370 started
Processing 367 finished
processing 371 started
Processing 369 finished
processing 372 started
Processing 370 finished
processing 373 started
Processing 373 finished
processing 374 started
Processing 37

Processing 525 finished
processing 528 started
Processing 528 finished
processing 529 started
Processing 527 finished
processing 530 started
Processing 526 finished
processing 531 started
Processing 529 finished
processing 532 started
Processing 530 finished
processing 533 started
Processing 531 finished
processing 534 started
Processing 533 finished
processing 535 started
Processing 534 finished
processing 536 started
Processing 535 finished
processing 537 started
Processing 532 finished
processing 538 started
Processing 537 finished
processing 539 started
Processing 536 finished
processing 540 started
Processing 538 finished
processing 541 started
Processing 539 finished
processing 542 started
Processing 541 finished
processing 543 started
Processing 540 finished
processing 544 started
Processing 542 finished
processing 545 started
Processing 543 finished
processing 546 started
Processing 545 finished
processing 547 startedProcessing 544 finished

processing 548 started
Processing 54

Processing 699 finished
processing 702 started
Processing 700 finished
processing 703 started
Processing 701 finished
processing 704 started
Processing 703 finished
processing 705 started
Processing 702 finished
processing 706 started
Processing 704 finished
processing 707 started
Processing 706 finished
processing 708 started
Processing 705 finished
processing 709 started
Processing 707 finished
processing 710 started
Processing 708 finished
processing 711 started
Processing 709 finished
processing 712 started
Processing 710 finished
processing 713 started
Processing 712 finished
processing 714 started
Processing 711 finished
processing 715 started
Processing 713 finished
processing 716 started
Processing 714 finished
processing 717 started
Processing 715 finished
processing 718 started
Processing 717 finished
processing 719 started
Processing 718 finished
processing 720 started
Processing 716 finished
processing 721 started
Processing 719 finished
Processing 720 finished
processing 7

...

Processing 2715 finished
processing 2718 started
Processing 2716 finished
processing 2719 started
Processing 2717 finished
processing 2720 started
Processing 2718 finished
processing 2721 started
Processing 2719 finished
processing 2722 started
Processing 2720 finished
processing 2723 started
Processing 2722 finished
processing 2724 started
Processing 2721 finished
processing 2725 started
Processing 2723 finished
processing 2726 started
Processing 2724 finished
processing 2727 started
Processing 2725 finished
processing 2728 started
Processing 2726 finished
processing 2729 started
Processing 2728 finished
processing 2730 started
Processing 2727 finished
processing 2731 started
Processing 2730 finished
processing 2732 started
Processing 2729 finished
processing 2733 started
Processing 2731 finished
processing 2734 started
Processing 2732 finished
processing 2735 started
Processing 2734 finished
processing 2736 started
Processing 2733 finished
processing 2737 started
Processing 2736 fini

Processing 2882 finished
processing 2885 started
Processing 2884 finished
processing 2886 started
Processing 2883 finished
processing 2887 started
Processing 2885 finished
processing 2888 started
Processing 2888 finished
processing 2889 started
Processing 2886 finished
processing 2890 started
Processing 2887 finished
processing 2891 started
Processing 2889 finished

processing 2892 startedProcessing 2890 finished
processing 2893 started
Processing 2892 finished
processing 2894 started
Processing 2891 finished
processing 2895 started
Processing 2893 finished
processing 2896 started
Processing 2895 finished
processing 2897 started
Processing 2894 finished
processing 2898 started
Processing 2896 finished
processing 2899 started
Processing 2897 finished
processing 2900 started
Processing 2898 finished
processing 2901 started
Processing 2899 finished
processing 2902 started
Processing 2900 finished
processing 2903 started
Processing 2901 finished
processing 2904 started
Processing 2902 fini

Processing 3049 finished
processing 3052 started
Processing 3050 finished
processing 3053 started
Processing 3051 finished
processing 3054 started
Processing 3053 finished
processing 3055 started
Processing 3052 finished
processing 3056 started
Processing 3054 finished
processing 3057 started
Processing 3056 finished
processing 3058 started
Processing 3055 finished
processing 3059 started
Processing 3057 finished
processing 3060 started
Processing 3058 finished
processing 3061 started
Processing 3059 finished
processing 3062 started
Processing 3060 finished
processing 3063 started
Processing 3061 finished
processing 3064 started
Processing 3062 finished
processing 3065 started
Processing 3063 finished
processing 3066 started
Processing 3064 finished
processing 3067 started
Processing 3065 finished
processing 3068 started
Processing 3066 finished
processing 3069 started
Processing 3067 finished
processing 3070 started
Processing 3068 finished
processing 3071 started
Processing 3069 fini

Processing 3216 finished
processing 3219 started
Processing 3217 finished
processing 3220 started
Processing 3218 finished
processing 3221 started
Processing 3219 finished
processing 3222 started
Processing 3220 finished
processing 3223 started
Processing 3222 finished
processing 3224 started
Processing 3221 finished
processing 3225 started
Processing 3223 finished
processing 3226 started
Processing 3225 finished
processing 3227 started
Processing 3226 finished
processing 3228 started
Processing 3227 finished
processing 3229 started
Processing 3224 finished
processing 3230 started
Processing 3228 finished
processing 3231 started
Processing 3229 finished
processing 3232 started
Processing 3230 finished
processing 3233 started
Processing 3231 finished
processing 3234 started
Processing 3232 finished
processing 3235 started
Processing 3233 finished
processing 3236 started
Processing 3234 finished
processing 3237 started
Processing 3235 finished
processing 3238 started
Processing 3237 fini

Processing 3383 finished
processing 3386 started
Processing 3384 finished
processing 3387 started
Processing 3385 finished
processing 3388 started
Processing 3386 finished
processing 3389 started
Processing 3387 finished
processing 3390 started
Processing 3388 finished
processing 3391 started
Processing 3389 finished
processing 3392 started
Processing 3390 finished
processing 3393 started
Processing 3391 finished
processing 3394 started
Processing 3392 finished
processing 3395 started
Processing 3393 finished
processing 3396 started
Processing 3395 finished
processing 3397 started
Processing 3394 finished
processing 3398 started
Processing 3396 finished
processing 3399 started
Processing 3397 finished
processing 3400 started
Processing 3398 finished
processing 3401 started
Processing 3399 finished
processing 3402 started
Processing 3400 finished
processing 3403 started
Processing 3401 finished
processing 3404 started
Processing 3402 finished
processing 3405 started
Processing 3403 fini

In [ ]:
# print(len(output))
# print(sum([len(o) for o in output]))

In [ ]:
# print(len(output))
# positive_sentences = [o for o in output if "label" in o]
# negative_sentences = [o for o in output if "label" not in o]
# print(len(positive_sentences))
# print(len(negative_sentences))

In [ ]:
# positive_output_file = os.path.join(base_loc, output_folder, "pos.json")
# negative_output_file = os.path.join(base_loc, output_folder, "neg.json")

# json.dump(positive_sentences, open(positive_output_file, 'w'))
# json.dump(negative_sentences, open(negative_output_file, 'w'))
